<a href="https://colab.research.google.com/github/hmghaly/arabic_tools/blob/master/kaust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/ar_challenges' #directory where we keep the data
os.chdir(cwd)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils


Cloning into 'code_utils'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 240 (delta 30), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (240/240), 965.81 KiB | 6.57 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [ ]:
#Get the training data
!wget https://wti-solveml.kaust.edu.sa/Uploads/DataSets/496b0bb7-4c7a-4831-8708-026918cea2b5/csv
#!wget https://wti-solveml.kaust.edu.sa/Uploads/DataSets/684ff648-a1ae-42a4-ba3b-daba98467105/csv

In [31]:
#Step 1 -
#Create BERT feature extraction function
!pip install transformers==3.1.0
import torch
import transformers
import itertools

bert_model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

align_layer=8
test_sent="وأنا راجع رحت أجيب الخضار"
sent_src=test_sent.strip().split()
token_src= [tokenizer.tokenize(word) for word in sent_src] #to use the bert tokenizer
#print(token_src)
#token_src= [[word] for word in sent_src] #just to use the original tokenization
wid_src= [tokenizer.convert_tokens_to_ids(x) for x in token_src]
ids_src = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids']
sent_bert_features = bert_model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

#sent_bert_features=sent2bert(test_sent)
print(sent_bert_features.shape)

torch.Size([13, 768])


In [32]:
#Step 2-
#create an RNN/LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

torch.manual_seed(1)
random.seed(1)

device = torch.device('cuda')


class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size,num_layers, matching_in_out=False, batch_size=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.batch_size = batch_size
    self.matching_in_out = matching_in_out #length of input vector matches the length of output vector 
    self.lstm = nn.LSTM(input_size, hidden_size,num_layers)
    self.hidden2out = nn.Linear(hidden_size, output_size)
    self.hidden = self.init_hidden()
  def forward(self, feature_list):
    feature_list.to(device) #### <<<<<<<<<<<<<<<<< 
    if self.matching_in_out:
      # test=feature_list.view(len( feature_list), 1, -1)
      # print(test.shape)
      lstm_out, _ = self.lstm( feature_list.view(len( feature_list), 1, -1))
      output_space = self.hidden2out(lstm_out.view(len( feature_list), -1))
      output_scores = torch.sigmoid(output_space) #we'll need to check if we need this sigmoid
      return output_scores #output_scores
    else:
      for i in range(len(feature_list)):
        cur_ft_tensor=feature_list[i]#.view([1,1,self.input_size])
        cur_ft_tensor=cur_ft_tensor.view([1,1,self.input_size])
        lstm_out, self.hidden = self.lstm(cur_ft_tensor, self.hidden)
        outs=self.hidden2out(lstm_out)
      return outs
  def init_hidden(self):
    #return torch.rand(self.num_layers, self.batch_size, self.hidden_size)
    return (torch.rand(self.num_layers, self.batch_size, self.hidden_size).to(device),
            torch.rand(self.num_layers, self.batch_size, self.hidden_size).to(device))

# test_sent="الناس دول مش عارفين بيعملوا إيه"
# sent_bert_features=sent2bert(test_sent)
# print(sent_bert_features.shape)

n_input=768
n_output=3
n_hidden=64
n_layers=3
io_matching=False

if io_matching:
  print("input size matches output size")
else:
  print("output size is fixed by n_output")


rnn = RNN(n_input, n_hidden, n_output,n_layers,io_matching)
rnn.to(device)
# sent_bert_features=sent_bert_features.to(device)
# print(sent_bert_features)
sent_bert_features=torch.rand((15, n_input)).to(device)
output = rnn(sent_bert_features)
output.to(device)
print("output.shape:", output.shape)
print("output:", output.view([n_output]))
print(rnn)

# features=[[0.0552, 0.5666, 0.0039, 0.0709, 0.9022, 0.2247],
#         [0.4827, 0.5223, 0.9601, 0.6596, 0.7441, 0.2861],
#         [0.3286, 0.8637, 0.6833, 0.6254, 0.1229, 0.1010],
#         [0.6010, 0.1248, 0.8506, 0.9166, 0.2929, 0.1777],
#         [0.9840, 0.2332, 0.1015, 0.1371, 0.5869, 0.5971]]

# #feature_tensor=torch.tensor(features)

# # print("shape of feature tensor", feature_tensor.shape)
# # print(feature_tensor)
# feature_tensor=torch.rand((15, n_input))
# # print("feature_tensor:", feature_tensor.shape)
# print(feature_tensor)
# #line_tensor=torch.rand((10, 4))
# output = rnn(sent_bert_features)
# print("output.shape:", output.shape)
# print("output:", output.view([n_output]))

# print(rnn)

output size is fixed by n_output
output.shape: torch.Size([1, 1, 3])
output: tensor([-0.0573,  0.0137, -0.0831], device='cuda:0', grad_fn=<ViewBackward>)
RNN(
  (lstm): LSTM(768, 64, num_layers=3)
  (hidden2out): Linear(in_features=64, out_features=3, bias=True)
)


In [33]:
#Step 3- Now defining the functions we will need
import os
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import * 
from itertools import groupby
from collections import Counter
import h5py


def sent2bert(sent):
  align_layer = 8
  sent_src=sent.strip().split()
  token_src= [tokenizer.tokenize(word) for word in sent_src] #to use the bert tokenizer
  #print(token_src)
  #token_src= [[word] for word in sent_src] #just to use the original tokenization
  wid_src= [tokenizer.convert_tokens_to_ids(x) for x in token_src]
  ids_src = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids']
  out_src = bert_model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  return out_src

def feature_extraction_full(raw_sent):
  sent_str=raw_sent.replace("<br>"," ")
  sent_str=clean_ar(sent_str)
  sent_toks=tok(sent_str)
  sent_toks_str=" ".join(sent_toks)
  feature_tensor=sent2bert(sent_toks_str)
  return feature_tensor

import gensim
aravec_model = gensim.models.Word2Vec.load('full_uni_cbow_300_twitter.mdl')
def feature_extraction_wv(raw_sent,wv_model):
  sent_str=raw_sent.replace("<br>"," ")
  sent_str=clean_ar(sent_str)
  sent_toks=tok(sent_str)
  for tok0 in sent_toks:
    toc_vector=wv_model[tok0]
    print(tok0,)  


ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def outcome2tensor(cur_outcome_label,all_outcome_labels):
  initial_list=[0.]*len(all_outcome_labels)
  initial_list[all_outcome_labels.index(cur_outcome_label)]=1.
  return torch.tensor(initial_list)


def pred2label(output_tensor,label_list):
  output_tensor=output_tensor.view([len(label_list)])
  output_list=output_tensor.tolist()
  #print(output_list, output_tensor, output_tensor.shape)
  pred_list=[(lb,output_list[i_]) for i_,lb in enumerate(label_list)]
  pred_list.sort(key=lambda x:-x[1])
  return pred_list

def pred_correctness_dist(full_preds_list):
  pred_dist=[]
  full_preds_list.sort()
  for key, group in groupby(full_preds_list,lambda x:x[0]):
    #print(key,Counter([v[1] for v in list(group)])) 
    pred_dist.append((key,Counter([v[1] for v in list(group)])))
  return pred_dist

print("loaded the functions for feature and outcome processing")


loaded the functions for feature and outcome processing


In [5]:
#Step 4 - 
#now get the data and split to training and dev sets
#use both the WTI training and sarcasm training for a larger datase

import h5py
#hdf5_fpath=os.path.join(exp_dir,"data.hdf5")
hdf5_fpath="data.hdf5"
hdf5_file=h5py.File(hdf5_fpath, 'w')
train_grp = hdf5_file.create_group("train")
sarcasm_grp = hdf5_file.create_group("sarcasm")
test_grp = hdf5_file.create_group("test")
   # for item in hdf5_file: 
    #     obj=hdf5_file[item]
    #     for ds in obj:
    #         cur_data=obj[ds]
    #         print(item, ds, cur_data.shape, cur_data.attrs["uas"])
        #print(item)
# dset = grp.create_dataset(example_id, data=one_hot_tensor_torch, compression="gzip")
# dset.attrs["uas"]=parser_uas
# dset.attrs["sent_size"]=sent_size        



import pandas as pd
#first the WTI training
full_training_list=[]
training_fopen=open("combined_training.tsv")
for i0,tf_line in enumerate(training_fopen):
  if i0%100==0: print("training item:", i0)
  line_split=tf_line.strip("\n\r\t").split("\t")
  if len(line_split)!=3: continue
  tweet_id,sentiment_str,tweet_text=line_split
  if sentiment_str.lower()=="positive": sentiment=1
  elif sentiment_str.lower()=="negative": sentiment=-1
  else: sentiment=0
  feature_tensor=feature_extraction_full(tweet_text)
  ft_numpy=feature_tensor.detach().numpy()
  dset = train_grp.create_dataset(str(tweet_id), data=ft_numpy, compression="gzip")
  dset.attrs["text"]=tweet_text
  dset.attrs["sentiment"]=sentiment        

  full_training_list.append((tweet_text,sentiment))
  #if i0>20: break
#hdf5_file.close()

#Now for the data of sarcasm
train_df = pd.read_csv("ArSarcasm_train.csv")
for i,a in enumerate(train_df.iterrows()):
  if i%50==0: print("sarcasm item: ",i)
  tweet_id="sarcasm_%s"%i
  row_dict=dict(iter(a[1].items())) 
  #print(row_dict)
  tweet_text=row_dict["tweet"]
  sentiment_str=row_dict["sentiment"] # print(row_dict["dialect"])
  if sentiment_str.lower()=="positive": sentiment=1
  elif sentiment_str.lower()=="negative": sentiment=-1
  else: sentiment=0
  full_training_list.append((tweet_text,sentiment))
  feature_tensor=feature_extraction_full(tweet_text)
  ft_numpy=feature_tensor.detach().numpy()
  dset = sarcasm_grp.create_dataset(tweet_id, data=ft_numpy, compression="gzip")
  dset.attrs["text"]=tweet_text
  dset.attrs["sentiment"]=sentiment      
  #if i>20: break  


dev_set=full_training_list[:5000]
train_set=full_training_list[5000:]
print("full training size", len(full_training_list))
print("full dev_set", len(dev_set))
print("full train_set", len(train_set))

print("now doing the test set")
test_set_with_ids=[]
test_df = pd.read_csv("test1_with_text.csv")
for i,a in enumerate(test_df.iterrows()):
  rnn.zero_grad()
  if i%20==0: print("test item", i)
  row_dict=dict(iter(a[1].items())) 
  #print(row_dict)
  tweet_id=row_dict["Tweet_id"]
  tweet_text=row_dict["Text"]
  test_set_with_ids.append((tweet_id,tweet_text))
  feature_tensor=feature_extraction_full(tweet_text)
  ft_numpy=feature_tensor.detach().numpy()
  dset = test_grp.create_dataset(str(tweet_id), data=ft_numpy, compression="gzip")
  dset.attrs["text"]=tweet_text
  #dset.attrs["sentiment"]=sentiment  
  #if i>20: break      

print("test_set_with_ids",len(test_set_with_ids))
hdf5_file.close()

training item: 0
training item: 100
training item: 200
training item: 300
training item: 400
training item: 500
training item: 600
training item: 700
training item: 800
training item: 900
training item: 1000
training item: 1100
training item: 1200
training item: 1300
training item: 1400
training item: 1500
training item: 1600
training item: 1700
training item: 1800
training item: 1900
training item: 2000
training item: 2100
training item: 2200
training item: 2300
training item: 2400
training item: 2500
training item: 2600
training item: 2700
training item: 2800
training item: 2900
training item: 3000
training item: 3100
training item: 3200
training item: 3300
training item: 3400
training item: 3500
training item: 3600
training item: 3700
training item: 3800
training item: 3900
training item: 4000
training item: 4100
training item: 4200
training item: 4300
training item: 4400
training item: 4500
training item: 4600
training item: 4700
training item: 4800
training item: 4900
training ite

In [ ]:
sent="مش متأكد بصراحة"
test_tensor=feature_extraction_full(sent)
test_tensor_numpy=test_tensor.detach().numpy()
print(test_tensor.shape)
print(test_tensor_numpy.shape)

torch.Size([9, 768])
(9, 768)


In [ ]:
#Step 5 - start training the RNN/LSTM
import os
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import * 
from itertools import groupby
from collections import Counter
import h5py



torch.manual_seed(1)
random.seed(1)

outcomes=[-1,0,1]

exp_name="try_64_2layers_full_new"
models_dir="models"
exp_dir=os.path.join(models_dir,exp_name)
if not os.path.exists(exp_dir): os.makedirs(exp_dir)

n_input=768
n_output=3
n_hidden=64
n_layers=2
io_matching=False
rnn = RNN(n_input, n_hidden, n_output,n_layers,io_matching)
rnn.to(device)
print(rnn)

loss_func = nn.MSELoss()
LR=0.00001
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters


n_epochs=50


hdf5_fpath="data.hdf5"
hdf5_file=h5py.File(hdf5_fpath, 'r')

train_grp=hdf5_file["train"]
dev_keys=list(train_grp.keys())[:2000]
train_keys=list(train_grp.keys())[2000:]
test_grp=hdf5_file["test"]
test_keys=list(test_grp.keys())

for ep0 in range(n_epochs):
  print("epoch #",ep0)
  model_fname="%s-%s.model"%(exp_name,ep0)
  PATH=os.path.join(exp_dir,model_fname)
  if os.path.exists(PATH):
    checkpoint = torch.load(PATH)
    rnn.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    rnn.train()
    print("model found, proceeding to next epoch")
    continue
    #rnn_model.to(device)    


  train_loss=0
  dev_loss=0
  train_counter=0
  dev_counter=0  
  dev_n_correct=0
  dev_preds_list=[]
  #for sent_str,sentiment in train_set[:1000]:
  for t_key in train_keys:
    if train_counter%100==0: print(train_counter)
    cur_hd_item=train_grp[t_key]
    sentiment=cur_hd_item.attrs["sentiment"]
    text=cur_hd_item.attrs["text"]
    cur_feature_tensor=torch.tensor(cur_hd_item)
    cur_feature_tensor=cur_feature_tensor.to(device)

    # print(cur_hd_item.attrs["sentiment"])
    # print(cur_hd_item.attrs["text"])
    # print(torch.tensor(cur_hd_item))
    # #print(dir(cur_hd_item))
    # continue
    
    # #print(sent_str,sentiment)
    # cur_feature_tensor=feature_extraction_full(sent_str)
    correct_output_tensor=outcome2tensor(sentiment,outcomes)
    correct_output_tensor=correct_output_tensor.to(device)
    rnn.hidden = rnn.init_hidden()
    rnn.zero_grad()
    rnn_output=rnn(cur_feature_tensor)
    rnn_output=rnn_output.to(device)
    #print(rnn_output.shape)
    #print("RNN output:", rnn_output.view([n_output]))
    #print("Correct output:",correct_output_tensor)
    correct_output_tensor=correct_output_tensor.view(rnn_output.shape)
    loss = loss_func(rnn_output, correct_output_tensor)
    #print("current loss:", loss.item())
    train_loss+=loss.item()
    train_counter+=1
    loss.backward()
    optimizer.step()
  print("now working on dev_set")
  #for sent_str,sentiment in dev_set[:100]:
  for dev_key in dev_keys:
    if dev_counter%100==0: print(dev_counter)
    cur_hd_item=train_grp[dev_key]
    sentiment=cur_hd_item.attrs["sentiment"]
    text=cur_hd_item.attrs["text"]
    cur_feature_tensor=torch.tensor(cur_hd_item)
    cur_feature_tensor=cur_feature_tensor.to(device)

    #print(sent_str,sentiment)
    #cur_feature_tensor=feature_extraction_full(sent_str)
    correct_output_tensor=outcome2tensor(sentiment,outcomes)
    correct_output_tensor=correct_output_tensor.to(device)
    rnn.hidden = rnn.init_hidden()
    rnn.zero_grad()
    rnn_output=rnn(cur_feature_tensor)
    rnn_output=rnn_output.to(device)
    correct_output_tensor=correct_output_tensor.view(rnn_output.shape)
    loss = loss_func(rnn_output, correct_output_tensor)
    dev_loss+=loss.item()
    dev_counter+=1
    cur_pred_list=pred2label(rnn_output,outcomes)
    top_pred=cur_pred_list[0][0]
    is_correct=0
    if top_pred==sentiment: 
      dev_n_correct+=1
      is_correct=1
    dev_preds_list.append((sentiment,is_correct))
    #print("predicted:", cur_pred_list, "correct: ",sentiment)

  #continue
  print("total training loss:",train_loss,"count:",train_counter,"average loss:", train_loss/train_counter)
  avg_train_loss=train_loss/train_counter
  print("total dev loss:",dev_loss,"count:",dev_counter,"average loss:", dev_loss/dev_counter)
  avg_dev_loss=dev_loss/dev_counter
  dev_accuracy=round(dev_n_correct/dev_counter,4)
  cur_pred_dist=pred_correctness_dist(dev_preds_list)
  print(cur_pred_dist)
  # print(Counter(all_preds_list))
  # all_preds_list.sort()
  print("n_correct:",dev_n_correct, "prediction accuracy:", round(dev_n_correct/dev_counter,4))
  print("---------")

  torch.save({
            'epoch': ep0,
            'model_state_dict': rnn.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'dev_loss': avg_dev_loss,
            'dev_accuracy': dev_accuracy, 
            'pred_dist': cur_pred_dist,
            'n_input': n_input,
            'n_output': n_output,
            'n_hidden': n_hidden,
            'n_layers': n_layers, 
            'outcomes': outcomes,
            'io_matching':io_matching,
            'LR':LR

            }, PATH)  
  #outcome_1hot=tensor.torch([])
# print("now working on the test set")
# prediction_list=[]
# test_i=0
# #for tweet_id,sent_str in test_set_with_ids:
# for test_key0 in test_keys:
#   if test_i%100==0: print(test_i)
#   cur_hd_item=test_grp[test_key0]
#   #sentiment=cur_hd_item.attrs["sentiment"]
#   text=cur_hd_item.attrs["text"]
#   cur_feature_tensor=torch.tensor(cur_hd_item)  
#   # continue
  
#   # #print(sent_str,sentiment)
#   # cur_feature_tensor=feature_extraction_full(sent_str)
#   rnn.hidden = rnn.init_hidden()
#   rnn.zero_grad()
#   rnn_output=rnn(cur_feature_tensor)
#   cur_pred_list=pred2label(rnn_output,outcomes)
#   top_pred=cur_pred_list[0][0]
#   prediction_list.append((tweet_id,top_pred))
#   test_i+=1
#   # print(tweet_id,top_pred,sent_str)
#   # print("-------")

# import csv
# cur_fname="submission4.csv"  
# # field names 
# fields = ['Tweet_id', 'sentiment']    
# # data rows of csv file 
# rows= prediction_list
# with open(cur_fname, 'w') as f:
#     # using csv.writer method from CSV package
#     write = csv.writer(f)
#     write.writerow(fields)
#     write.writerows(rows)
hdf5_file.close()

RNN(
  (lstm): LSTM(768, 64, num_layers=2)
  (hidden2out): Linear(in_features=64, out_features=3, bias=True)
)
epoch # 0
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
1

In [9]:
#Step 6 - Now predicting using our best models
import os, h5py
import pandas as pd

outcomes=[-1,0,1]

exp_name="try_64_3_full_new"
models_dir="models"
exp_dir=os.path.join(models_dir,exp_name)
if not os.path.exists(exp_dir): os.makedirs(exp_dir)
epoch=18
model_fname="%s-%s.model"%(exp_name,epoch)
PATH=os.path.join(exp_dir,model_fname)

checkpoint = torch.load(PATH)
n_input=checkpoint['n_input']
n_output=checkpoint['n_output']
n_hidden=checkpoint['n_hidden']
n_layers=checkpoint['n_layers']
io_matching=False
rnn_model = RNN(n_input, n_hidden, n_output,n_layers,io_matching)
rnn_model.load_state_dict(checkpoint['model_state_dict'])
rnn_model.to(device)
#rnn.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print(rnn_model)
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# n_input=768
# n_output=3
# n_hidden=64
# n_layers=3
# io_matching=False
# rnn = RNN(n_input, n_hidden, n_output,n_layers,io_matching)


# print(checkpoint.keys())
for key,val in checkpoint.items():
  if "dict" in key: continue
  print(key,val)
#rnn_model.hidden = rnn_model.init_hidden()
#rnn_model.zero_grad()
rnn_model.eval()

# test_tensor=torch.rand([12, 768]).to(device)
# raw_sent="أنا مش متأكد إيه اللي هيحصل هنا"
# sent_str=raw_sent.replace("<br>"," ")
# sent_str=clean_ar(sent_str)
# sent_toks=tok(sent_str)
# sent_toks_str=" ".join(sent_toks)
# feature_tensor=sent2bert(sent_toks_str)

#feature_tensor=feature_extraction_full(test_str).to(device)
#out=model(feature_tensor)
#out=model(test_tensor)
# def predict(sent_str):
#   feature_tensor=feature_extraction_full(sent_str).to(device)
#   rnn_out=rnn_model(feature_tensor)
#   print(rnn_out)

# test="أنا مش متأكد إيه اللي هيحصل هنا"
# predict(test)
# test="إيه الزفت دة"
# predict(test)

hdf5_fpath="data.hdf5"
hdf5_file=h5py.File(hdf5_fpath, 'r')
test_grp=hdf5_file["test"]
test_keys=list(test_grp.keys())
test_df = pd.read_csv("test1_with_text.csv")
prediction_list=[]
for i,a in enumerate(test_df.iterrows()):
  rnn_model.init_hidden()
  rnn_model.zero_grad()
  #if i>50: break
  if i%100==0: print("test item", i)
  row_dict=dict(iter(a[1].items())) 
  #print(row_dict)
  tweet_id=str(row_dict["Tweet_id"])
  #print(tweet_id)
  tweet_text=row_dict["Text"]
  cur_hd_item=test_grp[tweet_id]
  #print(cur_hd_item.attrs["text"])
  cur_feature_tensor=torch.tensor(cur_hd_item).to(device)
  #print(tweet_text)
  #cur_feature_tensor=feature_extraction_full(tweet_text).to(device)
  rnn_out=rnn_model(cur_feature_tensor)
  cur_pred_list=pred2label(rnn_out,outcomes)
  top_pred=cur_pred_list[0][0]

  # if i<1000:
  #   cur_feature_tensor=feature_extraction_full(tweet_text).to(device)
  #   rnn_out=rnn_model(cur_feature_tensor)
  #   cur_pred_list=pred2label(rnn_out,outcomes)
  #   top_pred=cur_pred_list[0][0]
  # else:
  #   if "زفت" in tweet_text: top_pred=-1
  #   else: top_pred=0

    
  #print(cur_pred_list)
  prediction_list.append((tweet_id,top_pred))
  #test_i+=1



# print("now working on the test set")
# prediction_list=[]
# test_i=0
# #for tweet_id,sent_str in test_set_with_ids:
# for test_key0 in test_keys:
#   if test_i%100==0: print(test_i)
#   cur_hd_item=test_grp[test_key0]
#   #sentiment=cur_hd_item.attrs["sentiment"]
#   text=cur_hd_item.attrs["text"]
#   cur_feature_tensor=torch.tensor(cur_hd_item)  
#   # continue
  
#   # #print(sent_str,sentiment)
#   # cur_feature_tensor=feature_extraction_full(sent_str)
#   rnn.hidden = rnn.init_hidden()
#   rnn.zero_grad()
#   rnn_output=rnn(cur_feature_tensor)
#   cur_pred_list=pred2label(rnn_output,outcomes)
#   top_pred=cur_pred_list[0][0]
#   prediction_list.append((tweet_id,top_pred))
#   test_i+=1
#   # print(tweet_id,top_pred,sent_str)
#   # print("-------")

import csv
cur_fname="submission8.csv"  
# field names 
fields = ['Tweet_id', 'sentiment']    
# data rows of csv file 
rows= prediction_list
with open(cur_fname, 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(rows)



hdf5_file.close()


RNN(
  (lstm): LSTM(768, 64, num_layers=3)
  (hidden2out): Linear(in_features=64, out_features=3, bias=True)
)
epoch 18
train_loss 0.06731816923196385
dev_loss 0.1894536556837859
dev_accuracy 0.632
pred_dist [(-1, Counter({0: 258, 1: 103})), (0, Counter({1: 885, 0: 109})), (1, Counter({0: 369, 1: 276}))]
n_input 768
n_output 3
n_hidden 64
n_layers 3
outcomes [-1, 0, 1]
io_matching False
LR 1e-05
test item 0
test item 100
test item 200
test item 300
test item 400
test item 500
test item 600
test item 700
test item 800
test item 900
test item 1000
test item 1100
test item 1200
test item 1300
test item 1400
test item 1500
test item 1600
test item 1700
test item 1800
test item 1900
test item 2000
test item 2100
test item 2200
test item 2300
test item 2400
test item 2500
test item 2600
test item 2700
test item 2800
test item 2900
test item 3000
test item 3100
test item 3200
test item 3300
test item 3400
test item 3500
test item 3600
test item 3700
test item 3800
test item 3900
test item 400

In [8]:
import os
import torch
exp_name="try_64_3_full_new"
models_dir="models"
exp_dir=os.path.join(models_dir,exp_name)
if not os.path.exists(exp_dir): os.makedirs(exp_dir)
for epoch in range(50):
  
  model_fname="%s-%s.model"%(exp_name,epoch)
  PATH=os.path.join(exp_dir,model_fname)
  if not os.path.exists(PATH): continue
  print(epoch)
  checkpoint = torch.load(PATH)
  print("train_loss", checkpoint["train_loss"])
  print("dev_loss", checkpoint["dev_loss"])
  print("pred_dist", checkpoint["pred_dist"])
  print("dev_accuracy", checkpoint["dev_accuracy"])
  print("--------")
 

0
train_loss 0.13290933960558338
dev_loss 0.19821556433735169
pred_dist [(-1, Counter({0: 358, 1: 3})), (0, Counter({1: 972, 0: 22})), (1, Counter({0: 510, 1: 135}))]
dev_accuracy 0.555
--------
1
train_loss 0.11855566232499826
dev_loss 0.1852338074836007
pred_dist [(-1, Counter({0: 355, 1: 6})), (0, Counter({1: 960, 0: 34})), (1, Counter({0: 406, 1: 239}))]
dev_accuracy 0.6025
--------
2
train_loss 0.11369803979148257
dev_loss 0.1807393021238651
pred_dist [(-1, Counter({0: 310, 1: 51})), (0, Counter({1: 930, 0: 64})), (1, Counter({0: 404, 1: 241}))]
dev_accuracy 0.611
--------
3
train_loss 0.10989065623901201
dev_loss 0.1793657651357039
pred_dist [(-1, Counter({0: 289, 1: 72})), (0, Counter({1: 921, 0: 73})), (1, Counter({0: 412, 1: 233}))]
dev_accuracy 0.613
--------
4
train_loss 0.10680578951682725
dev_loss 0.17788268529832477
pred_dist [(-1, Counter({0: 279, 1: 82})), (0, Counter({1: 912, 0: 82})), (1, Counter({0: 411, 1: 234}))]
dev_accuracy 0.614
--------
5
train_loss 0.103951116

In [ ]:
def predict(sent_str):
  feature_tensor=feature_extraction_full(sent_str)
  rnn_out=rnn(feature_tensor)
  return rnn_out
  #print(rnn_out)

test="أنا مش متأكد إيه اللي هيحصل هنا"
predict(test)

tensor([[[0.1649, 0.7413, 0.1047]]], grad_fn=<AddBackward0>)

In [ ]:
import pandas as pd
import requests

#op=requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=5)
train_df = pd.read_csv("training_file.csv")
limit=40000
all_ids=[]
for i,a in enumerate(train_df.iterrows()):
  #print(a)
  tw_id=str(a[1].Tweet_id)
  sentiment=a[1].sentiment
  all_ids.append(tw_id)

  # print(">>>",tw_id, sentiment)
  # if i>30: break
  # url='https://twitter.com/i/web/status/'+tw_id
  # print(url)
  # op=requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=5)
  # html_content=op.text
print(len(all_ids))
first_part="\n".join([str(v) for v in all_ids[:40000]])
second_part="\n".join([str(v) for v in all_ids[40000:]])
fopen=open("first.csv","w")
fopen.write(first_part)
fopen.close()
fopen=open("second.csv","w")
fopen.write(second_part)
fopen.close()

55000


In [ ]:
#Run this code to combine the two parts of the WTI training data
import pandas as pd
import json
train_df = pd.read_csv("training_file.csv")
all_ids=[]
sentiment_dict={}
for i,a in enumerate(train_df.iterrows()):
  #print(a)
  tw_id=str(a[1].Tweet_id)
  sentiment=a[1].sentiment
  sentiment_dict[tw_id]=sentiment
  all_ids.append(tw_id)

first_fopen=open("first.json")
first_content=first_fopen.read()
first_fopen.close()
first_list=json.loads(first_content)

second_fopen=open("second.json")
second_content=second_fopen.read()
second_fopen.close()
second_list=json.loads(second_content)

print(len(first_list), len(second_list))

combined_fopen=open("combined_training.tsv","w")
combined=first_list+second_list
for a in combined:
  cur_id=a["id"]
  cur_txt=a["text"]
  cur_txt=cur_txt.replace("\n","<br>").replace("\r","<br>").replace("\t"," ")
  cur_sentiment=sentiment_dict[cur_id]
  line="%s\t%s\t%s\n"%(cur_id,cur_sentiment,cur_txt)
  combined_fopen.write(line)
  # print(cur_id,cur_txt,cur_sentiment)
  # print("----")
combined_fopen.close()

24614 8595


In [ ]:
for s in second_list[:100]:
  print(s["text"])
  print("-----")

@SaudiHousingCC 
هل احتساب قيمة دعم القرض تكون
الراتب الاساسي + بدل السكن مخصوم منه مستحقات التامينات ؟؟
-----
@SaudiHousing لم يتم ايداع الدعم السكني في حسابي هذا الشهر
-----
@ejar_sa السلام عليكم متى يتم شحن الرصيد بعد سداد الفاتوره بالبنك
-----
صراحة مرا مووقتكم  حجرصحي وشبكة زفت مرا نبغى حل جذري.    @STC
-----
المافيا في ايطاليا تبرعت للحكومه  الإيطالية 10 مليار دولار لمواجهة  فيروس كورونا ..فيكى الخير يا مافيا✋😂
-----
@ejar_sa  السلام عليكم حبيت اسال هل يلزم المستاجر بدفع مبلغ مالي من أجل اصدار العقد الإلكتروني
-----
@majedhogail الاكثريه مانزل لهم الدعم نتمنا توضيح السبب علما بأن الاشهر السابقه ينزل ولا في اي سبب علما بأنه وصلت رساله من معاليك
-----
المفروض من باب الادب ان
@stc
@Mobily
@Zain
يفتحو تطبيقات مكالمات الفيديو مجانا خلال فترة منع التجول على الاقل

وضعية الميت قبيحه
-----
@SaudiHousing السلام عليكم
هل يمكن الحصول على القرض الحسن الخاص بمنسوبي التعليم بدون القرض المدعوم
-----
#أمر_ملكي: تكليف ماجد الحقيل وزير الإسكان بالقيام بمهام وزير الشؤون البلدية والقروية بالإضافة إل

In [ ]:
import pandas as pd
import requests

#processing UNTerm entries
#from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *


#op=requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=5)
train_df = pd.read_csv("ArSarcasm_train.csv")
limit=40000
all_ids=[]
for i,a in enumerate(train_df.iterrows()):
  row_dict=dict(iter(a[1].items())) 
  #print(row_dict)
  tweet_txt=row_dict["tweet"]
  tweet_txt=tweet_txt.replace("_"," _ ")
  tweet_txt_clean=clean_ar(tweet_txt)
  print(row_dict["sentiment"])
  print(row_dict["dialect"])
  print(tok(tweet_txt_clean))

  print("========")
  if i>100: break
  # tw_id=str(a[1].Tweet_id)
  # sentiment=a[1].sentiment
  # all_ids.append(tw_id)


negative
gulf
['"', 'نصيحه', 'ما', 'عمرك', 'اتنزل', 'لعبة', 'سوبر', 'ماريو', 'مش', 'زي', 'ما', 'كنا', 'متوقعين', 'الله', 'يرحم', 'ايامات', 'السيقا', 'والفاميلي', '#', 'SuperMarioRun', '"']
neutral
msa
['"', '#', 'نادين', '_', 'نسيب', '_', 'نجيم', '❤', '️', '❤', '️', '❤', '️', 'مجلة', '#', 'ماري', '_', 'كلير', '💭', '#', 'ملكة', '_', 'الصحراء', '👑', 'https', ':', '/', '/', 't', '.', 'co', '/', 'fVtn489TUS', '@', 'nadinenjeim', '@', 'MarieClaire', '_', 'AR', '"']
neutral
egypt
['"', '@', 'Alito', '_', 'NBA', 'اتوقع', 'انه', 'بيستمر', '"']
neutral
levant
['"', '@', 'KSA24', 'يعني', '"', 'بموافقتنا', '"', 'لأن', 'دمشق', 'صايرة', 'موسكو', '"']
neutral
msa
['"', 'RT', '@', 'alaahmad20', ':', 'قائد', 'في', 'الحرس', 'يعترف', 'بفقدان', 'السيطرة', 'الأمنية', 'في', 'شرقي', 'وغربي', 'إيران', '-', 'أحوازنا', '#', 'الأحواز', '#', 'السعودية', '@', 'abo', '_', 'asseel', '@', 'spagov', 'https', ':', '/', '…', '"']
negative
egypt
['شوال', 'الفلوس', 'سويرس', 'مشغول', 'اوى', 'اليومين', 'دول', 'بقناة', 'الج

In [ ]:
#now let's prepare a submission
import pandas as pd
import requests

import os
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import * 
from itertools import groupby
from collections import Counter

ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

#op=requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=5)
test_df = pd.read_csv("test1_with_text.csv")
#limit=40000
all_ids=[]
prediction_list=[]
sentiment_list=[]
for i,a in enumerate(test_df.iterrows()):
  rnn.zero_grad()
  if i%20==0: print(i)
  row_dict=dict(iter(a[1].items())) 
  #print(row_dict)
  tweet_id=row_dict["Tweet_id"]
  tweet_txt=row_dict["Text"]
  tweet_txt=tweet_txt.replace("_"," _ ")
  tweet_txt_clean=clean_ar(tweet_txt)
  tmp_words=tok(tweet_txt_clean)
  ar_words=tok_ar(tmp_words,ar_counter_dict)
  #print(tweet_txt_clean)
  sentiment=0
  for word in ar_words:
    if "زفت" in word: sentiment=-1
    if "يلعن" in word: sentiment=-1
    if "ملعون" in word: sentiment=-1
    if "سيء" in word: sentiment=-1
    if "أكره" in word: sentiment=-1
    if word.split("_")[-1] in ["خرا","خراء","خرائي"]: sentiment=-1
    if "فخور" in word: sentiment=1
    # if "زين" in word: sentiment=1
    #if "عظيم" in word: sentiment=1
  rnn_output=predict(tweet_txt)
  cur_pred_list=pred2label(rnn_output,outcomes)
  sentiment=cur_pred_list[0][0]

  # print(tweet_txt)
  # print(pred)
  # print("------")
  
  # if sentiment==1:
  #   print(tweet_txt_clean)
  #   print("-----")
  prediction_list.append((tweet_id,sentiment))
  sentiment_list.append(sentiment)

print(Counter(sentiment_list))
#now writing the predictions
import csv

cur_fname="submission2.csv"  
# field names 
fields = ['Tweet_id', 'sentiment']    
# data rows of csv file 
rows= prediction_list
with open(cur_fname, 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(rows)


# rows = [ ['Nikhil', 'COE', '2', '9.0'], 
#          ['Sanchit', 'COE', '2', '9.1'], 
#          ['Aditya', 'IT', '2', '9.3'], 
#          ['Sagar', 'SE', '1', '9.5'], 
#          ['Prateek', 'MCE', '3', '7.8'], 
#          ['Sahil', 'EP', '2', '9.1']] 
  


In [ ]:
#Let's use BERT multilingual model to extract the embeddings
import itertools
src="I do understand nothing"
src="أنا مش فاهم حاجة"
src="ربما الموت يقترب مني وانا لا اشعر به لطفك يا الله في سكرة موتي ان تكون خاتمتي حسنه ثم الجنه"
align_layer = 8

sent_src=src.strip().split()
token_src= [tokenizer.tokenize(word) for word in sent_src] #to use the bert tokenizer
#token_src= [[word] for word in sent_src] #just to use the original tokenization
wid_src= [tokenizer.convert_tokens_to_ids(x) for x in token_src]
ids_src = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids']
out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

print("token_src",token_src)
print("wid_src",wid_src)
print("ids_src", ids_src)
print(out_src.shape)


token_src [['ر', '##ب', '##ما'], ['ال', '##موت'], ['ي', '##قت', '##رب'], ['من', '##ي'], ['و', '##انا'], ['لا'], ['اش', '##عر'], ['به'], ['ل', '##ط', '##ف', '##ك'], ['يا'], ['الله'], ['في'], ['س', '##كرة'], ['موت', '##ي'], ['ان'], ['تكون'], ['خ', '##ات', '##مت', '##ي'], ['حسن', '##ه'], ['ثم'], ['ال', '##جن', '##ه']]
wid_src [[773, 11086, 14495], [59901, 67538], [793, 39053, 30075], [10289, 10461], [791, 37641], [13879], [92118, 69861], [10327], [787, 14286, 11687, 12497], [60844], [15764], [10210], [775, 62708], [83131, 10461], [14269], [24027], [770, 10564, 34783, 10461], [27304, 10388], [15902], [59901, 56760, 10388]]
ids_src tensor([  101,   773, 11086, 14495, 59901, 67538,   793, 39053, 30075, 10289,
        10461,   791, 37641, 13879, 92118, 69861, 10327,   787, 14286, 11687,
        12497, 60844, 15764, 10210,   775, 62708, 83131, 10461, 14269, 24027,
          770, 10564, 34783, 10461, 27304, 10388, 15902, 59901, 56760, 10388,
          102])
torch.Size([39, 768])


wid_src [[146], [10149], [49151]]
ids_src tensor([  101,   146, 10149, 49151,   102])
torch.Size([3, 768])


In [ ]:
import h5py
hdf5_fpath=os.path.join(exp_dir,"data.hdf5")
hdf5_file=h5py.File(hdf5_fpath, 'w')
grp = hdf5_file.create_group(it)
   # for item in hdf5_file: 
    #     obj=hdf5_file[item]
    #     for ds in obj:
    #         cur_data=obj[ds]
    #         print(item, ds, cur_data.shape, cur_data.attrs["uas"])
        #print(item)
dset = grp.create_dataset(example_id, data=one_hot_tensor_torch, compression="gzip")
dset.attrs["uas"]=parser_uas
dset.attrs["sent_size"]=sent_size        
hdf5_file.close()


In [ ]:
device_num=1

if device_num==0: device_name="cuda:0"
elif device_num==1: device_name="cuda:1"
else: device_name="cpu"


#device = torch.device(device_name if torch.cuda.is_available() else "cpu")
#device = torch.device('/device:GPU:0')
#device = torch.device('device:GPU:0')
device = torch.device('cuda')

embeds=embeds.to(device)
def init_hidden(self):
  return (torch.zeros(1, self.batch_size, self.hidden_size).to(device),
      torch.zeros(1, self.batch_size, self.hidden_size).to(device)) 
rnn = RNN(n_input, n_hidden, 1)
rnn.to(device)

category_tensor=torch.tensor(correct_heads, dtype=torch.float)
category_tensor=category_tensor.to(device)
output = rnn(line_tensor)
output=output.to(device)

In [11]:
#testing how to get the data from the hd5 files
import h5py

hdf5_fpath="data.hdf5"
hdf5_file=h5py.File(hdf5_fpath, 'r')
train_grp=hdf5_file["train"]
dev_keys=list(train_grp.keys())[:5000]
train_keys=list(train_grp.keys())[5000:]
test_grp=hdf5_file["test"]
test_keys=list(test_grp.keys())

print("number of training items:", len(train_keys))
print("dev_set size:",len(dev_keys))
print("number of test items:", len(test_keys))

# for it in train_grp.items():
#   print(it, it[1].attrs["text"])

  #print(dir(it[1]))
#print(dir(train_grp))
#print()
#print(hdf5_file.items())
hdf5_file.close()

number of training items: 28209
dev_set size: 5000
number of test items: 20000


In [10]:
#Clone the github codebase for aravec
import os
if os.path.exists("aravec"): os.removedirs("aravec")
#!rm -r aravec #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/bakrianoo/aravec.git aravec



Cloning into 'aravec'...
remote: Enumerating objects: 142, done.
remote: Total 142 (delta 0), reused 0 (delta 0), pack-reused 142
Receiving objects: 100% (142/142), 1.20 MiB | 11.69 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [14]:
!wget https://bakrianoo.s3-us-west-2.amazonaws.com/aravec/full_uni_cbow_300_twitter.zip

--2021-03-29 01:38:56--  https://bakrianoo.s3-us-west-2.amazonaws.com/aravec/full_uni_cbow_300_twitter.zip
Resolving bakrianoo.s3-us-west-2.amazonaws.com (bakrianoo.s3-us-west-2.amazonaws.com)... 52.218.233.57
Connecting to bakrianoo.s3-us-west-2.amazonaws.com (bakrianoo.s3-us-west-2.amazonaws.com)|52.218.233.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2833686412 (2.6G) [application/zip]
Saving to: ‘full_uni_cbow_300_twitter.zip’

full_uni_cbow_300_t 100%[===================>]   2.64G  30.3MB/s    in 97s     

2021-03-29 01:40:33 (27.8 MB/s) - ‘full_uni_cbow_300_twitter.zip’ saved [2833686412/2833686412]



In [15]:
!unzip full_uni_cbow_300_twitter.zip

Archive:  full_uni_cbow_300_twitter.zip
  inflating: full_uni_cbow_300_twitter.mdl  
  inflating: full_uni_cbow_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_uni_cbow_300_twitter.mdl.wv.vectors.npy  


In [ ]:
import gensim
t_model = gensim.models.Word2Vec.load('full_uni_cbow_300_twitter.mdl')


In [29]:
token="الحين"
most_similar = t_model.wv.most_similar( token, topn=10 )
for term, score in most_similar:
    print(term, score)
print(t_model[token])

اللحين 0.9532021880149841
والحين 0.7758309841156006
احين 0.7538288831710815
هالحين 0.7515158653259277
ذحين 0.7432119846343994
للحين 0.7227292060852051
خخ 0.7090663313865662
دحين 0.7004846334457397
بعدين 0.6678630113601685
الحيين 0.6590395569801331
[-5.39580643e-01 -2.04826093e+00  8.97866905e-01 -7.31150985e-01
 -1.73954618e+00  1.60691619e+00 -4.47108299e-01  2.23364145e-01
  4.74283636e-01 -3.82652432e-01 -5.82857907e-01 -1.82154012e+00
  1.59363902e+00 -3.91100824e-01 -1.13415682e+00 -9.23327327e-01
  2.40160418e+00  7.57520497e-01 -2.47649050e+00  2.27496552e+00
  9.63017046e-01  1.03695735e-01 -1.95540929e+00  2.17612967e-01
 -3.01175956e-02  5.87216020e-01  8.87227416e-01  6.85424864e-01
 -2.84710860e+00  3.86379504e+00  1.89357913e+00  9.91025090e-01
  1.64410770e+00  1.73927629e+00  2.16164804e+00  6.65032715e-02
 -1.91086078e+00 -9.18030024e-01 -2.53077698e+00 -2.38934255e+00
 -1.60595167e+00 -1.31678939e-01  6.78059638e-01 -4.64854777e-01
 -3.97334844e-01  9.75031316e-01  2.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

#data = pd.read_excel(r"C:\Users\ibrom\Desktop\NOTEBOOK\NLP PRACICE\AJGT.xlsx")

#data = pd.read_table("combined_training.tsv")
data = pd.read_csv("combined_training_copy.tsv", sep='\t')
print("head", data.head())
print(data.sample(5))

punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
  
data['Feed'] = data['Feed'].apply(preprocess)
print(data.head(5))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
head              ID Sentiment                                               Feed
0  1.221875e+18  Positive  @nas_alharbi8 والله حسب الأرقام سيكون مخيب للآ...
1  1.221884e+18   Neutral  الزعل بيغير ملامحك ، بيغير نظرة العين ، بيغير ...
2  1.226423e+18  Positive  الحب الحقيقي هو اقتسام بعض نفسك مع شخص أخر أقر...
3  1.221881e+18  Positive                          @Mo_Fat7 النهضة في فتيل 😂
4  1.221884e+18   Neutral  @halgawi @DmfMohe ليس حباً في ايران بقدر ماهو ...
                 ID  ...                                               Feed
6464   1.182925e+18  ...  @ALAKBArKWWT مالومكم والله .. <br>اكيد زعلانين...
19134  1.221784e+18  ...  أقوى قوة ممكن أن تملكها<br>هي وجودك بأفكارك في...
14302  1.215176e+18  ...  @STCcare_KSA السلام عليكم عندي شريحه زين<br> ا...
8761   1.146669e+18  ...  انتي ايش اللي يريحك تبي تجي عريانه يعني ؟ http...
6048   1.223399e+18  ...  هيجى علي

In [20]:
# splitting the data into target and feature
feature = data.Feed
target = data.Sentiment
# splitting into train and tests
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

# make pipeline
pipe = make_pipeline(TfidfVectorizer(),
                    LogisticRegression())
# make param grid
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

# create and fit the model
lr_model = GridSearchCV(pipe, param_grid, cv=5)
lr_model.fit(X_train,Y_train)

# make prediction and print accuracy
prediction = lr_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy score is 0.74
              precision    recall  f1-score   support

    Negative       0.63      0.24      0.35       982
     Neutral       0.76      0.95      0.84      4549
    Positive       0.62      0.27      0.38       989

    accuracy                           0.74      6520
   macro avg       0.67      0.49      0.53      6520
weighted avg       0.72      0.74      0.70      6520



In [21]:
test_df = pd.read_csv("test1_with_text.csv")

test_df["Text"] = test_df["Text"].apply(preprocess)
test_df.head(5)


,Tweet_id,Text
0,1251608623798722560,kamnapp bipksa nazahagovsa MCgovSA شوفوا هالاج...
1,1221535162539872256,ربما الموت يقترب مني وانا اشعر لطفك الله سكره ...
2,1252203833385779200,mhrsdcare السلام عليكم لوسمحت تحديث ملف المنشا...
3,1221884276636938240,لقد نام تاركا حلمه وداءع الله فايقظه الله علي ...
4,1252686719461273602,nahdihope طالبه طلبيه منكم امس ولحد الان ماوصل...


In [30]:
test_df["Prediction"] = lr_model.predict(test_df["Text"])
test_list=list(test_df["Prediction"])
print(len([v for v in test_list if v=="Negative"]))
prediction_list=[]
for i,a in enumerate(test_df.iterrows()):
  if i%100==0: print("test item", i)
  row_dict=dict(iter(a[1].items())) 
  tweet_id=str(row_dict["Tweet_id"])
  tweet_text=row_dict["Text"]
  cur_pred=row_dict["Prediction"]
  top_pred=0
  if cur_pred=="Positive": top_pred=1
  if cur_pred=="Negative": top_pred=-1

  #print(cur_pred, tweet_id)

  #top_pred=cur_pred_list[0][0]

  prediction_list.append((tweet_id,top_pred))



import csv
cur_fname="submission10sk.csv"  
# field names 
fields = ['Tweet_id', 'sentiment']    
# data rows of csv file 
rows= prediction_list
with open(cur_fname, 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(rows)


1394
test item 0
test item 100
test item 200
test item 300
test item 400
test item 500
test item 600
test item 700
test item 800
test item 900
test item 1000
test item 1100
test item 1200
test item 1300
test item 1400
test item 1500
test item 1600
test item 1700
test item 1800
test item 1900
test item 2000
test item 2100
test item 2200
test item 2300
test item 2400
test item 2500
test item 2600
test item 2700
test item 2800
test item 2900
test item 3000
test item 3100
test item 3200
test item 3300
test item 3400
test item 3500
test item 3600
test item 3700
test item 3800
test item 3900
test item 4000
test item 4100
test item 4200
test item 4300
test item 4400
test item 4500
test item 4600
test item 4700
test item 4800
test item 4900
test item 5000
test item 5100
test item 5200
test item 5300
test item 5400
test item 5500
test item 5600
test item 5700
test item 5800
test item 5900
test item 6000
test item 6100
test item 6200
test item 6300
test item 6400
test item 6500
test item 6600
te

In [18]:
pipe = make_pipeline(TfidfVectorizer(),
                    RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
             'randomforestclassifier__max_features':['sqrt', 'log2']}

rf_model = GridSearchCV(pipe, param_grid, cv=5)
rf_model.fit(X_train,Y_train)

prediction = rf_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")

KeyboardInterrupt: ignored

In [19]:
pipe = make_pipeline(TfidfVectorizer(),
                    MultinomialNB())
pipe.fit(X_train,Y_train)
prediction = pipe.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.70
              precision    recall  f1-score   support

    Negative       0.93      0.01      0.03       982
     Neutral       0.70      1.00      0.82      4549
    Positive       0.77      0.01      0.02       989

    accuracy                           0.70      6520
   macro avg       0.80      0.34      0.29      6520
weighted avg       0.75      0.70      0.58      6520

